$$
\DeclareMathOperator\tr{tr}
\DeclareMathOperator\Tr{Tr}
\DeclareMathOperator\cof{cof}
\DeclareMathOperator\Cof{Cof}
\DeclareMathOperator\det{det}
\DeclareMathOperator\Det{Det}
\DeclareMathOperator\dev{dev}
\DeclareMathOperator\Dev{Dev}
\DeclareMathOperator\sph{sph}
\DeclareMathOperator\Sph{Sph}
\newcommand{\bF}{\mathbf{F}}
\newcommand{\bC}{\mathbf{C}}
\newcommand{\bE}{\mathbf{E}}
\newcommand{\bone}{\mathbf{1}}
\newcommand{\bd}{\mathbf{d}}
\newcommand{\bc}{\mathbf{c}}
\newcommand{\bA}{\mathbf{A}}
\newcommand{\bS}{\mathbf{S}}
\newcommand{\Bsigma}{{\boldsymbol{\sigma}}}
\newcommand{\pdiff}[2]{\frac{\partial#1}{\partial#2}}
\nonumber
$$

# Overview

In this notebook we numerically investigate different hyperelastic material laws.

# General setup

In [23]:
%matplotlib nbagg

In [24]:
from netgen import *
import netgen.occ as nocc
from netgen.webgui import Draw as DrawGeo

from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve import solvers

import matplotlib.pyplot as plt
import numpy as np

In [25]:
cube = nocc.Box((-0.5, -0.5, -0.5), (0.5, 0.5, 0.5))

In [26]:
cube.faces.Max(nocc.X).bc("front")
cube.faces.Min(nocc.X).bc("back")
cube.faces.Max(nocc.Y).bc("right")
cube.faces.Min(nocc.Y).bc("left")
cube.faces.Max(nocc.Z).bc("top")
cube.faces.Min(nocc.Z).bc("bottom")

In [27]:
ngmesh = nocc.OCCGeometry(cube).GenerateMesh(maxh=0.5)

In [28]:
mesh = Mesh(ngmesh)

In [29]:
Draw(mesh)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 3, 'order2d': 1, 'order3d':…

BaseWebGuiScene

In [30]:
order = 2

In [31]:
bc_boundaries = dict(
    dirichletx="back", 
    dirichlety="left", 
    dirichletz="bottom"
)

fes = VectorH1(mesh, order=order, **bc_boundaries)

In [32]:
#VectorH1?

In [33]:
u = GridFunction(fes) # stored displacement
u_trial = fes.TrialFunction() # displacement variable

# Material models

For a unified interface we employ the right Cauchy-Green tensor as basic input together with a parameter dictionary.


## Isotropy, invariants

The models under consideration describe isotropic materials. Hence, the strain energy densities 
must depends only on the "principal invariants", $I_1$, $I_2$ and $I_3$, of the right Cauchy-Green tensor
\begin{equation}
I_1 = \Tr \bC = ||\bF||^2, \quad I_2 = \Tr (\cof \bC) = ||\Cof\bF||^2 \quad \text{and} \quad I_3 = \Det \bC = \left(\Det \bF\right)^2 .
\end{equation}

An alternative expression for the second invariant is 
\begin{equation}
I_2 = \frac{1}{2}\left(\Tr(\bC)^2 - (I_1)^2\right).
\end{equation}

 
*Note*: The invariants as defined above need some more careful discussion when we know a bit more about differential geometry.

In [34]:
def phi(u):
    return u + (x, y, z)

def F(phi):
    return Grad(phi)

def Fu(u):
    return Grad(u) + Id(3)

def C(F):
    return F.trans * F

def I_1(A):
    return Trace(A)

def I_2(A):
    return (I_1(A * A) - I_1(A)) / 2

def I_3(A):
    return Det(A)

## St. Venant-Kirchhoff (SVK) material

The "St. Venant-Kirchhoff" material is the most straight-forward extension of Hooke's law in the setting of linearized elasticity to finite strains.
It's parameterization is based on the "Green-Lagrange" strain tensor that encodes the change of length as
\begin{equation}
\bE = \frac{1}{2} \left(\bC - \bone\right),
\end{equation}
where $\bC$ is the right Cauchy-Green tensor and $\bone$ the identity matrix.

The strain energy density function is given as
\begin{equation}
 \Psi(\bE) = \frac{\lambda}{2} (\Tr{\bE})^2 + {\mu} \Tr{\left(\bE^2\right)}.
\end{equation}

In [35]:
def Psi_SVK(C, **params):
    lmbda = params["lmbda"]
    mu = params["mu"]
    E = (C - Id(3)) / 2
    return lmbda/2 * Trace(E)**2 + mu * InnerProduct(E, E)

## Neo-Hookean (NH) material

This (class of) material model(s) is popular because of its theoretical properties and simplicity.
It often is a good choice for "moderate" deformations (strains in the order of 100%).

A compressible version reads
\begin{equation}
\Psi(\bC) = \frac{\mu}{2} \left(I_1 - 3 - \ln I_3\right) + \frac{\lambda}{8}\left(\ln I_3\right)^2 = 
\frac{\mu}{2} \left(I_1 - 3 - 2 \ln J\right) + \frac{\lambda}{2}\left(\ln J\right)^2,
\end{equation}
where $J = \Det\bF$.

In [36]:
def Psi_NH(C, **params):
    lmbda = params["lmbda"]
    mu = params["mu"]
    _I_1 = I_1(C)
    _ln_I_3 = log(I_3(C))
    return mu/2 * (_I_1 - 3 - _ln_I_3) + lmbda/8 * _ln_I_3**2

## A note on material parameters

Hyperelastic material models are often parameterized in terms of the *Lamé* parameters (or constants) $\lambda$ and $\mu$,
whereby $\mu$ is also known as the shear modulus $G$.
They are related to Young's modulus ("E-Modul") and Poisson's ratio via
\begin{equation}
\nu = \frac{\lambda}{2(\lambda + \mu)} \quad\text{and}\quad
E = 2\mu(1+\nu) .
\end{equation}

## Questions

 1. What are the expressions of the invariants in terms of the eigenvalues $\bF$ denoted as $\lambda_1$, $\lambda_2$ and $\lambda_3$?
 2. Consider the deformation map $\varphi$ for which $\varphi^i(X^1, X^2, X^3) = \varphi^i(X^i)$. What are the expressions for $\lambda_\alpha$ in terms of $\varphi$? (Note: the index of lambda is rather a mere label than an index indicating a component)
 3. In general, exponentiations of a tensor are computed by (i) transform to eigensystem, (ii) exponentiate eigenvalues and (iii) transform back. Why can $\bC^2$ be computed as $\bC^2 = \bC\cdot\bC$?
 4. How does the St. Venant-Kirchhoff (SVK) material read in terms of invariants of $\bC$?
 5. What is the expression for the second Piola-Kirchoff stress for the SVK model?
 6. What is the expression for the Cauchy stress for the NH model?
 7. What is the linearization of the Green-Lagrange strain tensor $\bE$ at $\bC = \bone$ expressed in terms of (incremental) displacements $(\Delta) u$? *Hint*: What special state does $\bC = \bone$ refer to?
 7. What are the *linearizations* (material tangents; derivative of "stress" wrt. "strain") of the of the SVK and the NH models at $\bC = \bone$?

# Formulation of the governing potential (energy)

In the present quasi-static settings, "equilibrium" is characterized by critical points of the total
energy.

Parameters to play with: 
 * loading direction/increment: sign/value of `dt`
 * material model: def. of `Psi = lambda...`

In [37]:
mu = Parameter(1)
lmbda = Parameter(5)
Psi = lambda C: Psi_SVK(C, mu=mu, lmbda=lmbda)

In [38]:
Pi = BilinearForm(fes, symmetric=True)

## Constitutive (material, internal) contribution

In [39]:
Pi += Variation(Psi(C(Fu(u_trial))) * dx(bonus_intorder=3))

## Loading (external) potential

In [40]:
t_3 = Parameter(0)
Pi += Variation(- t_3 * u_trial[2] * ds(mesh.Boundaries("top")))

# Solution procedure

In [41]:
# record some data
from pkgutil import read_code


t_3_data = []
u_top_data = []

# load increment 
dt = -0.01

# initialize
t_3.Set(0)
u.vec[:] = 0

r = u.vec.CreateVector()
Pi.Apply(u.vec, r)
print(f"initial residual norm = {r.Norm():e}")

if r.Norm() > 1e-14:
    raise RuntimeError("Initial residual is not zero!")

while t_3.Get() < 10 + 1e-6: # have some tolerance
    print(f"Solve for t_3 = {t_3.Get():e}")
    retcode, iterations = solvers.Newton(Pi, u, maxit=10, inverse="pardiso")
    if retcode != 0:
        raise RuntimeError("Solver failed")
    t_3_data.append(t_3.Get())
    u_top_data.append(u(mesh(0, 0, 0.5))[2])
    print(retcode)
    print(f"top displacement u_3 = {u_top_data[-1]:e}")
    print("-" * 120)
    t_3.Set(t_3.Get() + dt)

initial residual norm = 0.000000e+00
Solve for t_3 = 0.000000e+00
Newton iteration  0
err =  0.0
0
top displacement u_3 = 0.000000e+00
------------------------------------------------------------------------------------------------------------------------
Solve for t_3 = -1.000000e-02
Newton iteration  0
err =  0.005940885257859964
Newton iteration  1
err =  3.6356682182896376e-05
Newton iteration  2
err =  1.4431531964402412e-09
Newton iteration  3
err =  6.512355771998333e-16
0
top displacement u_3 = -3.548275e-03
------------------------------------------------------------------------------------------------------------------------
Solve for t_3 = -2.000000e-02
Newton iteration  0
err =  0.005972702794311201
Newton iteration  1
err =  3.716740977361688e-05
Newton iteration  2
err =  1.5273319833365094e-09
Newton iteration  3
err =  4.0522497872954227e-16
0
top displacement u_3 = -7.135004e-03
-------------------------------------------------------------------------------------------

RuntimeError: Solver failed

In [42]:
plt.plot(t_3_data, u_top_data)

<IPython.core.display.Javascript object>

## Questions

 1. How far can the SVK material be compressed?
 2. How far can the NH material be compressed?
 3. What is the key difference between the graphs?
 4. Do you suspect the "problem" to be rather in the material model or in the FE formulation/solution procedure?
 5. How would the code look like when the deformation map $\varphi$ is used instead of the displacement field $u$?